In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [67]:
data = pd.read_csv('H1b_EDA_data.csv',encoding = 'ISO-8859-1')
data.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMP_STATE_abb,...,WORKSITE_POSTAL_CODE,YEAR,NAIC_CODE,WAGE_RATE_OF_PAY,total_wage,Sector_data,EMP_STATE_full,EMP_State_and_city,Worksite_STATE_full,Worksite_State_and_city
0,I-200-09121-701936,WITHDRAWN,2/5/2015,2/5/2015,H-1B,02/09/2015,02/28/2015,"MEDTRONIC, INC.",MINNEAPOLIS,MN,...,55412,2015,3345.0,20000 -,19000.0,33.0,Minnesota,"MINNEAPOLIS, Minnesota",California,"EDEN PRAIRIE, California"
1,I-200-09146-796321,CERTIFIED,12/12/2014,12/18/2014,H-1B,01/05/2015,01/04/2018,UNIVERSITY OF OKLAHOMA,NORMAN,OK,...,73019,2015,6113.0,85000.00 -,42860.0,61.0,Oklahoma,"NORMAN, Oklahoma",Oklahoma,"NORMAN, Oklahoma"
2,I-200-09180-329758,WITHDRAWN,1/3/2013,12/15/2014,H-1B,01/07/2013,01/06/2016,NOKIA INC.,IRVING,TX,...,92127,2015,5172.0,94000 -,73965.0,51.0,Texas,"IRVING, Texas",California,"SAN DIEGO, California"
3,I-200-09183-259985,CERTIFIED,3/10/2015,3/16/2015,H-1B,09/07/2015,09/07/2018,"OMRON OILFIELD AND MARINE, INC.",HOUSTON,TX,...,77088,2015,3353.0,66000 - 70000,65998.0,33.0,Texas,"HOUSTON, Texas",Texas,"HOUSTON, Texas"
4,I-200-09189-800933,CERTIFIED,12/17/2014,12/23/2014,H-1B,06/06/2015,06/05/2018,FEDERAL HOME LOAN MORTGAGE CO.,MCLEAN,VA,...,22102,2015,5222.0,97000 - 143300,96907.0,52.0,Virginia,"MCLEAN, Virginia",Virginia,"MCLEAN, Virginia"


In [68]:
def count_na(column):
    return column.isna().sum()
data.apply(count_na)

CASE_NUMBER                     0
CASE_STATUS                     0
CASE_SUBMITTED                  1
DECISION_DATE                   0
VISA_CLASS                      0
EMPLOYMENT_START_DATE          38
EMPLOYMENT_END_DATE            47
EMPLOYER_NAME                  67
EMPLOYER_CITY                  29
EMP_STATE_abb                   0
EMPLOYER_POSTAL_CODE           57
EMPLOYER_COUNTRY                0
EMPLOYER_PHONE                  1
AGENT_ATTORNEY_NAME        227034
AGENT_ATTORNEY_CITY        840452
AGENT_ATTORNEY_STATE       899957
JOB_TITLE                      11
SOC_CODE                       18
SOC_NAME                       20
FULL_TIME_POSITION              7
PREVAILING_WAGE                 0
PW_UNIT_OF_PAY                  0
WAGE_UNIT_OF_PAY               19
WORKSITE_CITY                  41
WORKSITE_COUNTY              4616
Worksite_STATE_abb             16
WORKSITE_POSTAL_CODE           70
YEAR                            0
NAIC_CODE                       2
WAGE_RATE_OF_P

## Preprocessing

1. Delete the unused features. 
2. Calculate the duration of datetime. 
3. Delete the rows with NA. 
4. Do specific preprocessing listed in the following table. 


|Name|NACount|Note|
| :---- | :---- | :---- |
|Sector_data|2|<font color = 'red'>Delete </font>|
|EMP_STATE_full|2358|<font color = 'red'>Delete </font>|
|EMP_State_and_city|0|<font color = 'red'>Delete </font>|
|Worksite_STATE_full|2453|<font color = 'red'>Delete </font>|
|Worksite_State_and_city|0|<font color = 'red'>Delete </font>|
|EMPLOYER_PHONE|1|<font color = 'red'>Delete </font>|
|AGENT_ATTORNEY_NAME|227034|<font color = 'red'>Delete </font>|
|AGENT_ATTORNEY_CITY|840452|<font color = 'red'>Delete </font>|
|AGENT_ATTORNEY_STATE|899957|<font color = 'red'>Delete </font>|
|JOB_TITLE|11|<font color = 'red'>Delete </font>|
|EMPLOYER_POSTAL_CODE|57|<font color = 'red'>Delete </font>|
|WORKSITE_POSTAL_CODE|70|<font color = 'red'>Delete </font>|
|YEAR|0|<font color = 'red'>Delete </font>|
|WORKSITE_COUNTY|4616|<font color = 'red'>Delete </font>|
|WAGE_RATE_OF_PAY|0|<font color = 'red'>Delete </font>|
|CASE_STATUS|0|<font color = 'green'>label </font>|
|CASE_SUBMITTED|1| <font color = 'blue'> calculate duration </font>|
|DECISION_DATE|0|<font color = 'blue'> calculate duration </font>|
|EMPLOYMENT_START_DATE|38|<font color = 'blue'> calculate duration </font>|
|EMPLOYMENT_END_DATE|47|<font color = 'blue'> calculate duration </font>|
|EMPLOYER_COUNTRY|0|<font color = 'blue'> Only consider American. </font>|
|PREVAILING_WAGE|0|<font color = 'blue'> Normalize it to yearly wage. </font>|
|PW_UNIT_OF_PAY|0|<font color = 'blue'> Normalize it to yearly wage. </font>|
|WAGE_UNIT_OF_PAY|19|<font color = 'blue'> Normalize it to yearly wage. </font>|
|total_wage|0|<font color = 'blue'> Normalize it to yearly wage. </font>|
|NAIC_CODE|2|<font color = 'blue'> Keep three digits. </font>|
|EMPLOYER_CITY|29|<font color = 'blue'> For geographical figures.  </font>|
|EMP_STATE_abb|0|<font color = 'blue'> For geographical figures. </font>|
|CASE_NUMBER|0||
|VISA_CLASS|0||
|EMPLOYER_NAME|67||
|SOC_CODE|18||
|SOC_NAME|20||
|FULL_TIME_POSITION|7||
|WORKSITE_CITY|41||
|Worksite_STATE_abb|16||


In [69]:
label = ['CASE_STATUS']
target = data.loc[:, label]

features = ['CASE_SUBMITTED', 'DECISION_DATE', 'EMPLOYMENT_START_DATE', 
            'EMPLOYMENT_END_DATE', 'EMPLOYER_COUNTRY', 'PREVAILING_WAGE', 
           'PW_UNIT_OF_PAY', 'WAGE_UNIT_OF_PAY', 'NAIC_CODE',
           'EMPLOYER_CITY', 'EMP_STATE_abb', 'CASE_NUMBER', 'VISA_CLASS',
           'EMPLOYER_NAME', 'SOC_CODE', 'SOC_NAME', 'FULL_TIME_POSITION',
           'WORKSITE_CITY', 'Worksite_STATE_abb', 'total_wage','CASE_STATUS']
train_vals = data.loc[:, features]

In [70]:
# Drop NA
train_vals = train_vals.dropna()
# Keep rows with employer country = usa
train_vals = train_vals[train_vals['EMPLOYER_COUNTRY'] == 'UNITED STATES OF AMERICA']

In [71]:
def naic_process(code):
    if(code > 100000):
        res = code // 1000
    elif(code > 10000):
        res = code // 100
    elif(code > 1000):
        res = code // 10
    else:
        res = code
    
    return str(int(code))

# Drop rows with NAIC_CODE less than 3 digits
train_vals = train_vals[train_vals['NAIC_CODE'] > 100]
# Keep the first 3 digits of the remaining NAIC_CODE
train_vals.loc[:, 'NAIC_CODE'] = train_vals.loc[:, 'NAIC_CODE'].apply(lambda x: naic_process(x))

In [72]:
import time 
import datetime
def str_to_datetime(string):
    try:
        timestamp = time.strptime(string, '%m/%d/%Y')
        return datetime.datetime(*timestamp[:6])
    except:
        try:
            timestamp = time.strptime(string, '%d/%m/%Y')
            return datetime.datetime(*timestamp[:6])
        except:
            return 

In [73]:
date_label = ['CASE_SUBMITTED', 
              'DECISION_DATE', 
              'EMPLOYMENT_START_DATE', 
              'EMPLOYMENT_END_DATE']
data_date = train_vals.loc[:,date_label]

In [74]:
data_date = data_date.applymap(str_to_datetime)

In [75]:
data_date.head()

,CASE_SUBMITTED,DECISION_DATE,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE
0,2015-02-05,2015-02-05,2015-02-09,2015-02-28
1,2014-12-12,2014-12-18,2015-01-05,2018-01-04
2,2013-01-03,2014-12-15,2013-01-07,2016-01-06
3,2015-03-10,2015-03-16,2015-09-07,2018-09-07
4,2014-12-17,2014-12-23,2015-06-06,2018-06-05


In [76]:
train_vals = train_vals.drop(columns = date_label)
train_vals = pd.concat([train_vals, data_date], axis=1)
train_vals.head()


,EMPLOYER_COUNTRY,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_UNIT_OF_PAY,NAIC_CODE,EMPLOYER_CITY,EMP_STATE_abb,CASE_NUMBER,VISA_CLASS,EMPLOYER_NAME,...,SOC_NAME,FULL_TIME_POSITION,WORKSITE_CITY,Worksite_STATE_abb,total_wage,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE
0,UNITED STATES OF AMERICA,19000.0,Year,Year,3345,MINNEAPOLIS,MN,I-200-09121-701936,H-1B,"MEDTRONIC, INC.",...,MECHANICAL ENGINEERS,Y,EDEN PRAIRIE,CA,19000.0,WITHDRAWN,2015-02-05,2015-02-05,2015-02-09,2015-02-28
1,UNITED STATES OF AMERICA,42860.0,Year,Year,6113,NORMAN,OK,I-200-09146-796321,H-1B,UNIVERSITY OF OKLAHOMA,...,"ENGINEERING TEACHERS, POSTSECONDARY",Y,NORMAN,OK,42860.0,CERTIFIED,2014-12-12,2014-12-18,2015-01-05,2018-01-04
2,UNITED STATES OF AMERICA,73965.0,Year,Year,5172,IRVING,TX,I-200-09180-329758,H-1B,NOKIA INC.,...,"COMPUTER OCCUPATIONS, ALL OTHER*",Y,SAN DIEGO,CA,73965.0,WITHDRAWN,2013-01-03,2014-12-15,2013-01-07,2016-01-06
3,UNITED STATES OF AMERICA,65998.0,Year,Year,3353,HOUSTON,TX,I-200-09183-259985,H-1B,"OMRON OILFIELD AND MARINE, INC.",...,"ELECTRONICS ENGINEERS, EXCEPT COMPUTER",Y,HOUSTON,TX,65998.0,CERTIFIED,2015-03-10,2015-03-16,2015-09-07,2018-09-07
4,UNITED STATES OF AMERICA,96907.0,Year,Year,5222,MCLEAN,VA,I-200-09189-800933,H-1B,FEDERAL HOME LOAN MORTGAGE CO.,...,COMPUTER PROGRAMMERS,Y,MCLEAN,VA,96907.0,CERTIFIED,2014-12-17,2014-12-23,2015-06-06,2018-06-05


In [77]:
train_vals['CASE_DURATION'] = train_vals.loc[:,'DECISION_DATE'] - train_vals.loc[:,'CASE_SUBMITTED']
train_vals['EMPLOYMENT_DURATION'] = train_vals.loc[:,'EMPLOYMENT_END_DATE'] - train_vals.loc[:, 'EMPLOYMENT_START_DATE']

In [78]:
def to_days(e):
    try:
        return e.days
    except:
        return e

In [79]:
train_vals.loc[:,'CASE_DURATION'] = train_vals.loc[:,'CASE_DURATION'].apply(to_days) 
train_vals.loc[:, 'EMPLOYMENT_DURATION'] = train_vals.loc[:,'EMPLOYMENT_DURATION'].apply(to_days)
train_vals.head()


,EMPLOYER_COUNTRY,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_UNIT_OF_PAY,NAIC_CODE,EMPLOYER_CITY,EMP_STATE_abb,CASE_NUMBER,VISA_CLASS,EMPLOYER_NAME,...,WORKSITE_CITY,Worksite_STATE_abb,total_wage,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,CASE_DURATION,EMPLOYMENT_DURATION
0,UNITED STATES OF AMERICA,19000.0,Year,Year,3345,MINNEAPOLIS,MN,I-200-09121-701936,H-1B,"MEDTRONIC, INC.",...,EDEN PRAIRIE,CA,19000.0,WITHDRAWN,2015-02-05,2015-02-05,2015-02-09,2015-02-28,0,19.0
1,UNITED STATES OF AMERICA,42860.0,Year,Year,6113,NORMAN,OK,I-200-09146-796321,H-1B,UNIVERSITY OF OKLAHOMA,...,NORMAN,OK,42860.0,CERTIFIED,2014-12-12,2014-12-18,2015-01-05,2018-01-04,6,1095.0
2,UNITED STATES OF AMERICA,73965.0,Year,Year,5172,IRVING,TX,I-200-09180-329758,H-1B,NOKIA INC.,...,SAN DIEGO,CA,73965.0,WITHDRAWN,2013-01-03,2014-12-15,2013-01-07,2016-01-06,711,1094.0
3,UNITED STATES OF AMERICA,65998.0,Year,Year,3353,HOUSTON,TX,I-200-09183-259985,H-1B,"OMRON OILFIELD AND MARINE, INC.",...,HOUSTON,TX,65998.0,CERTIFIED,2015-03-10,2015-03-16,2015-09-07,2018-09-07,6,1096.0
4,UNITED STATES OF AMERICA,96907.0,Year,Year,5222,MCLEAN,VA,I-200-09189-800933,H-1B,FEDERAL HOME LOAN MORTGAGE CO.,...,MCLEAN,VA,96907.0,CERTIFIED,2014-12-17,2014-12-23,2015-06-06,2018-06-05,6,1095.0


In [80]:
train_vals = train_vals.dropna()
train_vals = train_vals.drop(columns='EMPLOYER_COUNTRY')
train_vals = train_vals.drop(columns=date_label)
train_vals.head()


,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_UNIT_OF_PAY,NAIC_CODE,EMPLOYER_CITY,EMP_STATE_abb,CASE_NUMBER,VISA_CLASS,EMPLOYER_NAME,SOC_CODE,SOC_NAME,FULL_TIME_POSITION,WORKSITE_CITY,Worksite_STATE_abb,total_wage,CASE_STATUS,CASE_DURATION,EMPLOYMENT_DURATION
0,19000.0,Year,Year,3345,MINNEAPOLIS,MN,I-200-09121-701936,H-1B,"MEDTRONIC, INC.",17-2141.00,MECHANICAL ENGINEERS,Y,EDEN PRAIRIE,CA,19000.0,WITHDRAWN,0,19.0
1,42860.0,Year,Year,6113,NORMAN,OK,I-200-09146-796321,H-1B,UNIVERSITY OF OKLAHOMA,25-1032,"ENGINEERING TEACHERS, POSTSECONDARY",Y,NORMAN,OK,42860.0,CERTIFIED,6,1095.0
2,73965.0,Year,Year,5172,IRVING,TX,I-200-09180-329758,H-1B,NOKIA INC.,15-1799,"COMPUTER OCCUPATIONS, ALL OTHER*",Y,SAN DIEGO,CA,73965.0,WITHDRAWN,711,1094.0
3,65998.0,Year,Year,3353,HOUSTON,TX,I-200-09183-259985,H-1B,"OMRON OILFIELD AND MARINE, INC.",17-2072,"ELECTRONICS ENGINEERS, EXCEPT COMPUTER",Y,HOUSTON,TX,65998.0,CERTIFIED,6,1096.0
4,96907.0,Year,Year,5222,MCLEAN,VA,I-200-09189-800933,H-1B,FEDERAL HOME LOAN MORTGAGE CO.,15-1131,COMPUTER PROGRAMMERS,Y,MCLEAN,VA,96907.0,CERTIFIED,6,1095.0


In [81]:
train_vals.loc[:,'FULL_TIME_POSITION'] = (train_vals.loc[:,'FULL_TIME_POSITION'] == 'Y')
print(np.unique(train_vals['WAGE_UNIT_OF_PAY']))
print(np.unique(train_vals['PW_UNIT_OF_PAY']))


['Bi-Weekly' 'Hour' 'Month' 'Week' 'Year']
['Bi-Weekly' 'Hour' 'Month' 'Week' 'Year']


In [82]:
tmp_multiplier = {
    'Bi-Weekly': 25,
    'Hour': 50*5*8,
    'Month' : 12,
    'Week': 50,
    'Year': 1
}
def to_multiplier(e):
    return tmp_multiplier[e]
train_vals.loc[:,'PW_UNIT_OF_PAY'] = train_vals.loc[:,'PW_UNIT_OF_PAY'].apply(to_multiplier)
train_vals.loc[:,'WAGE_UNIT_OF_PAY'] = train_vals.loc[:,'WAGE_UNIT_OF_PAY'].apply(to_multiplier)
train_vals.loc[:, 'PREVAILING_WAGE'] = train_vals.loc[:, 'PREVAILING_WAGE'] *train_vals.loc[:,'PW_UNIT_OF_PAY']
train_vals.loc[:, 'total_wage'] = train_vals.loc[:, 'total_wage']* train_vals.loc[:,'WAGE_UNIT_OF_PAY']
train_vals = train_vals.drop(columns=['PW_UNIT_OF_PAY', 'WAGE_UNIT_OF_PAY'])
train_vals.head()


,PREVAILING_WAGE,NAIC_CODE,EMPLOYER_CITY,EMP_STATE_abb,CASE_NUMBER,VISA_CLASS,EMPLOYER_NAME,SOC_CODE,SOC_NAME,FULL_TIME_POSITION,WORKSITE_CITY,Worksite_STATE_abb,total_wage,CASE_STATUS,CASE_DURATION,EMPLOYMENT_DURATION
0,19000.0,3345,MINNEAPOLIS,MN,I-200-09121-701936,H-1B,"MEDTRONIC, INC.",17-2141.00,MECHANICAL ENGINEERS,True,EDEN PRAIRIE,CA,19000.0,WITHDRAWN,0,19.0
1,42860.0,6113,NORMAN,OK,I-200-09146-796321,H-1B,UNIVERSITY OF OKLAHOMA,25-1032,"ENGINEERING TEACHERS, POSTSECONDARY",True,NORMAN,OK,42860.0,CERTIFIED,6,1095.0
2,73965.0,5172,IRVING,TX,I-200-09180-329758,H-1B,NOKIA INC.,15-1799,"COMPUTER OCCUPATIONS, ALL OTHER*",True,SAN DIEGO,CA,73965.0,WITHDRAWN,711,1094.0
3,65998.0,3353,HOUSTON,TX,I-200-09183-259985,H-1B,"OMRON OILFIELD AND MARINE, INC.",17-2072,"ELECTRONICS ENGINEERS, EXCEPT COMPUTER",True,HOUSTON,TX,65998.0,CERTIFIED,6,1096.0
4,96907.0,5222,MCLEAN,VA,I-200-09189-800933,H-1B,FEDERAL HOME LOAN MORTGAGE CO.,15-1131,COMPUTER PROGRAMMERS,True,MCLEAN,VA,96907.0,CERTIFIED,6,1095.0


In [83]:
soc_code_column = train_vals['SOC_NAME']
invalid_SOC_NAME = train_vals['SOC_NAME'].apply(lambda x: x[0].isalpha() == False)
train_vals = train_vals.loc[~invalid_SOC_NAME,:]

In [84]:
invalid_SOC_CODE = train_vals['SOC_CODE'].apply(
    lambda x: x[0].isdigit() == False)
train_vals = train_vals.loc[~invalid_SOC_CODE, :]


In [85]:
print(np.unique(train_vals['SOC_NAME']))
print(np.unique(train_vals['SOC_CODE']))
print(sum(train_vals['SOC_CODE'].apply(lambda x: '.'in x)))
print(sum(train_vals['SOC_CODE'].apply(lambda x: '-' not in x)))


['ACCONTANTS AND AUDITORS' 'ACCOUNTANT' 'ACCOUNTANTA AND AUDITORS' ...
 'WOODWORKING MACHINE SETTERS, OPERATORS, AND' 'WRITERS AND AUTHORS'
 'ZOOLOGISTS AND WILDLIFE BIOLOGISTS']
['030' '1-1132' '10-1021.00' ... '54-1124' '69638.00' '71-2141']
12586
235


In [88]:
train_vals = train_vals.loc[train_vals['SOC_CODE'].apply(lambda x: '.' not in x),:]
train_vals = train_vals.loc[train_vals['SOC_CODE'].apply(
    lambda x: '-' in x), :]
train_vals = train_vals.loc[train_vals['SOC_CODE'].apply(
    lambda x: len(x) == 7), :]
len(np.unique(train_vals['SOC_CODE']))


680

In [93]:
count_set = {}
for employer in tqdm(train_vals['EMPLOYER_NAME']):
    count_set[employer] = count_set.get(employer, 0) + 1


100%|██████████| 2435102/2435102 [00:01<00:00, 1378056.68it/s]


In [94]:
count_values = []
for employer in tqdm(count_set):
    count_values.append((count_set[employer], employer))
count_values.sort()

100%|██████████| 169255/169255 [00:00<00:00, 1202196.96it/s]


In [105]:
tmp = 0
for i in range(1, len(count_values) + 1):
    if count_values[-i][0] < 100:
        break
    tmp += count_values[-i][0]
    if tmp > 0.7*len(train_vals):
        break
print(i, count_values[-i][0], tmp/len(train_vals))
employer_name = set([count_values[-j][1] for j in range(1, i)])
def handleEmployer(x):
    if x in employer_name:
        return x
    return 'Other'


train_vals.loc[:, 'EMPLOYER_NAME'] = train_vals.loc[:, 'EMPLOYER_NAME'].apply(handleEmployer)
print(len(np.unique(train_vals.loc[:, 'EMPLOYER_NAME'])))


3369 99 0.6591543187923955
3369


**The data is for EDA. For training, we need further processing on VISA_CLASS (one-hot formation), CASE_NUMBER (delete), Worksite_STATE_abb (one-hot formation), SOC_CODE (one-hot formation), NAIC_CODE (one-hot formation) and EMP_STATE_abb (one-hot formation).**

In [106]:
train_vals.to_csv('Preprocessing.csv')